# VOQC: A Verifed Optimizer for Quantum Circuits <img src="tutorial-files/umd.jpeg" width="100" align="right">

Welcome to the tutorial for VOQC! 

VOQC is a **verified optimizer for quantum circuits**, implemented and *formally verified* in the [Coq proof assistant](https://coq.inria.fr/). All VOQC optimizations are *guaranteed* to preserve the semantics of the original circuit, meaning that any optimized circuit produced by VOQC has the same behavior as the input circuit. VOQC was presented as a [distinguished paper at POPL 2021](https://arxiv.org/abs/1912.02250). All of our code is available freely online.
* The Coq definitions and proofs are available at [github.com/inQWIRE/SQIR](https://github.com/inQWIRE/SQIR).
* The OCaml library is available at [github.com/inQWIRE/mlvoqc](https://github.com/inQWIRE/mlvoqc) and can be installed with `opam install voqc`. 
* The Python bindings and tutorial are available at [github.com/inQWIRE/pyvoqc](https://github.com/inQWIRE/pyvoqc).

To run VOQC in Python, we (1) extract the verified Coq code to OCaml, (2) compile the extracted OCaml code to a library, (3) wrap the OCaml library in C, and (4) use Python's ctypes to make calls into the C library. This tutorial introduces the transformations available in VOQC through our Python interface. For convenience, we have wrapped VOQC inside a pass manager for IBM's [IBM's Qiskit framework](https://qiskit.org/documentation/getting_started.html), allowing us to take advantage of Qiskit's utilities for quantum programming. In this tutorial, we'll use Qiskit for building and printing circuits.

## Outline

* Setup
* Example
* Optimization API

## Setup

Follow the installaton instructions in the pyvoqc directory. (If you already have opam installed, this should be as simple as running `opam install voqc` and `./install.sh`.)

For more details and troubleshooting, see the [README](https://github.com/inQWIRE/pyvoqc/REAMDE.md) in the pyvoqc repository.

To begin the tutorial, import modules below from Qiskit and VOQC. 

In [1]:
# Click on this cell and press Ctrl+Enter or run it with the "Run" button
from qiskit import QuantumCircuit
from qiskit.qasm import pi
from qiskit.test.mock import FakeTenerife
from qiskit.transpiler import CouplingMap
from pyvoqc.qiskit.voqc_pass import voqc_pass_manager

## Example

`voqc_pass_manager` constructs a Qiskit pass manager that performs VOQC optimization and Qiskit circuit mapping, wrapped in VOQC's verified translation validation. It takes the following (optional) inputs:
* `opts`: list of VOQC optimizations to apply (see the list of optimizations below)
* `layout_method`: Qiskit method to use for layout (trivial, dense, noise_adaptive, or sabre)
* `routing_method`: Qiskit method to use for routing (basic, stochastic, lookahead, or sabre)
* `backend_properties`, `coupling_map`, `seed_transpiler`: options for Qiskit layout/routing, passed directly to Qiskit's functions

In the example here, we read in a quantum circuit from the file "tof_3_example.qasm" and then run it through our VOQC Qiskit pass.

In [3]:
# Read circuit from file
circ = QuantumCircuit.from_qasm_file("tutorial-files/tof_3_example.qasm")
print("Initial circui:")
print(circ)

# Create a VOQC pass manager that uses Qiskit's SABRE mapping to map to the 
# Tenerife architecture, followed by VOQC's "optimize" function
backend = FakeTenerife()
coupling_map = CouplingMap(couplinglist=backend.configuration().coupling_map)
vpm = voqc_pass_manager(post_opts=["optimize"], layout_method="sabre", routing_method="sabre", coupling_map=coupling_map)

# Run the pass manager
new_circ = vpm.run(circ)
print("\nFinal circuit:")
print(new_circ.count_ops()) # showing gate counts instead of the full circuit for brevity

Initial circui:
                    
q_0: ──■─────────■──
       │         │  
q_1: ──■─────────■──
       │         │  
q_2: ──┼────■────┼──
       │  ┌─┴─┐  │  
q_3: ──┼──┤ X ├──┼──
     ┌─┴─┐└─┬─┘┌─┴─┐
q_4: ┤ X ├──■──┤ X ├
     └───┘     └───┘

Final circuit:
OrderedDict([('h', 28), ('cx', 18), ('tdg', 9), ('t', 8), ('s', 2)])


## Optimization API

VOQC provide the following optimizations,: *not propagation*, *single-qubit gate cancellation*, *two-qubit gate cancellation*, *rotation merging*, *hadamard reduction*, and *single-qubit gate merging*.

**TODO**: list default ordering of optimizations

We provide a brief example of each optimization below. For more details see Section 4 of [our paper](https://arxiv.org/abs/1912.02250).

### Not Propagation

*Not propagation* commutes X (logical NOT) gates rightward through the circuit, cancelling them when possible. In the example below, the leftmost X gate propagates through the CNOT gate to become two X gates. The upper X gate then propagates through the H gate to become a Z, and the two lower X gates cancel.

In [4]:
circ = QuantumCircuit(2)
circ.x(0)
circ.cx(0, 1)
circ.h(0)
circ.x(1)

print("Before Optimization:")
print(circ)

vpm = voqc_pass_manager(post_opts=["not_propagation"])
new_circ = vpm.run(circ)

print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
     ┌───┐     ┌───┐
q_0: ┤ X ├──■──┤ H ├
     └───┘┌─┴─┐├───┤
q_1: ─────┤ X ├┤ X ├
          └───┘└───┘

After Optimization:
          ┌───┐┌───┐
q_0: ──■──┤ H ├┤ Z ├
     ┌─┴─┐└───┘└───┘
q_1: ┤ X ├──────────
     └───┘          


### Single-qubit Gate Cancellation

*Single-qubit gate cancellation* has the same "propagate-cancel" structure as not propagation, except that gates revert back to their original positions if they fail to cancel. In the example below, the upper leftmost T gate commutes through the control of the CNOT, combining with the upper rightmost T gate to become an S gate. The lower T gate commutes through the H; CNOT; H subcircuit to cancel with the Tdg gate.

In [5]:
circ = QuantumCircuit(2)
circ.t(0)
circ.t(1)
circ.h(1)
circ.cx(0, 1)
circ.h(1)
circ.t(0)
circ.tdg(1)

print("Before Optimization:")
print(circ)

vpm = voqc_pass_manager(post_opts=["cancel_single_qubit_gates"])
new_circ = vpm.run(circ)

print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
     ┌───┐          ┌───┐       
q_0: ┤ T ├───────■──┤ T ├───────
     ├───┤┌───┐┌─┴─┐├───┤┌─────┐
q_1: ┤ T ├┤ H ├┤ X ├┤ H ├┤ Tdg ├
     └───┘└───┘└───┘└───┘└─────┘

After Optimization:
               ┌───┐
q_0: ───────■──┤ S ├
     ┌───┐┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘


### Two-qubit Gate Cancellation

*Two-qubit gate cancellation* is similar to single-qubit gate cancellation, except that it aims to commute and cancel CNOT gates. In the circuit below, the first CNOT gate commutes through the second, to cancel with the third.

In [6]:
circ = QuantumCircuit(3)
circ.cx(0, 1)
circ.cx(0, 2)
circ.cx(0, 1)

print("Before Optimization:")
print(circ)

vpm = voqc_pass_manager(post_opts=["cancel_two_qubit_gates"])
new_circ = vpm.run(circ)

print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
                    
q_0: ──■────■────■──
     ┌─┴─┐  │  ┌─┴─┐
q_1: ┤ X ├──┼──┤ X ├
     └───┘┌─┴─┐└───┘
q_2: ─────┤ X ├─────
          └───┘     

After Optimization:
          
q_0: ──■──
       │  
q_1: ──┼──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘


### Rotation Merging

*Rotation merging* combines Rz gates that act on the same logical state (see discussion in Sec 4.4 of [our paper](https://arxiv.org/abs/1912.02250)). In the example below, the two Rz(pi/6) gates can be combines into a single Rz(pi/6 + pi/6) = Rz(pi/3) gate.

In [7]:
circ = QuantumCircuit(2)
circ.rz(pi/6, 1)
circ.cx(0, 1)
circ.cx(1, 0)
circ.rz(pi/6, 0)

print("Before Optimization:")
print(circ)

vpm = voqc_pass_manager(post_opts=["merge_rotations"])
new_circ = vpm.run(circ)

print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
                     ┌───┐┌─────────┐
q_0: ─────────────■──┤ X ├┤ Rz(π/6) ├
     ┌─────────┐┌─┴─┐└─┬─┘└─────────┘
q_1: ┤ Rz(π/6) ├┤ X ├──■─────────────
     └─────────┘└───┘                

After Optimization:
                      ┌───┐
q_0: ──────────────■──┤ X ├
     ┌──────────┐┌─┴─┐└─┬─┘
q_1: ┤ Rz(1π/3) ├┤ X ├──■──
     └──────────┘└───┘     


### Hadamard Reduction

*Hadamard reduction* applies a series of identities to reduce the number of H gates in a circuit, for the purpose of making other optimizations (e.g. rotation merging) more effective. In the example below, we replace the first circuit with the second to remove two H gates.

In [8]:
circ = QuantumCircuit(2)
circ.h(1)
circ.sdg(1)
circ.cx(0, 1)
circ.s(1)
circ.h(1)

print("Before Optimization:")
print(circ)

vpm = voqc_pass_manager(post_opts=["hadamard_reduction"])
new_circ = vpm.run(circ)

print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
                                
q_0: ──────────────■────────────
     ┌───┐┌─────┐┌─┴─┐┌───┐┌───┐
q_1: ┤ H ├┤ Sdg ├┤ X ├┤ S ├┤ H ├
     └───┘└─────┘└───┘└───┘└───┘

After Optimization:
                      
q_0: ───────■─────────
     ┌───┐┌─┴─┐┌─────┐
q_1: ┤ S ├┤ X ├┤ Sdg ├
     └───┘└───┘└─────┘


### IBM Single-qubit Gate Merging

*Single-qubit gate merging* (called *optimize_ibm* in our code) combines adjacent single-qubit gates into a single gate. It will convert the input program to the gate set {U1, U2, U3, CX}.

In [9]:
circ = QuantumCircuit(1)
circ.h(0)
circ.x(0)
circ.s(0)

print("Before Optimization:")
print(circ)

vpm = voqc_pass_manager(post_opts=["optimize_ibm"])
new_circ = vpm.run(circ)

print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
   ┌───┐┌───┐┌───┐
q: ┤ H ├┤ X ├┤ S ├
   └───┘└───┘└───┘

After Optimization:
   ┌────────────┐
q: ┤ U2(π/2,2π) ├
   └────────────┘
